# Introduction to Decision Trees

A Decision Tree is a supervised machine learning algorithm used for both classification and regression tasks. It models decisions and their possible consequences, including chance event outcomes, resource costs, and utility. Decision Trees are easy to interpret, handle both numerical and categorical data, and can model complex relationships.

### Key Concepts

- **Nodes**: Each node represents a feature (attribute), and a condition is applied to that feature.
- **Root Node**: The top node in a decision tree, representing the best predictor.
- **Internal Nodes**: Nodes that have branches (child nodes).
- **Leaf Nodes (Terminal Nodes)**: Nodes that do not have any children. They represent the output class label (for classification) or a continuous value (for regression).
- **Branches**: Each branch represents the outcome of a decision or test. It connects nodes based on the outcome of a condition.
- **Splits**: The process of dividing a node into two or more sub-nodes based on a feature value. The goal is to create subsets of data that are as homogeneous as possible.
- **Entropy**: A measure of randomness or impurity in the dataset. Lower entropy means higher purity.




## Entropy and Information Gain

### Entropy
Entropy is a measure of randomness or impurity in the dataset. Lower entropy means higher purity.


### Information Gain (IG)
Information Gain is the reduction in entropy after a dataset is split on a feature. Higher information gain indicates a more effective feature for splitting.


In [17]:
Image(url='https://th.bing.com/th/id/OIP.pZo_Izi8-qfjunv43s5fzQAAAA?rs=1&pid=ImgDetMain')

### Steps to Build a Decision Tree

1. **Select the Best Feature**: Choose the feature that provides the highest information gain for splitting the dataset.
2. **Split the Data**: Divide the dataset into subsets based on the best feature.
3. **Create Decision Nodes**: Recursively split the subsets, creating decision nodes until the stopping criteria are met (maximum depth, minimum samples, or pure nodes).

### Stopping Criteria

- **Maximum Depth**: The tree stops growing when it reaches the specified maximum depth.
- **Minimum Samples per Split**: The tree stops growing if the number of samples in a node is less than the specified minimum.
- **Pure Nodes**: The tree stops splitting if all samples in a node belong to the same class.
 

# Decision Tree

In [14]:
import numpy as np
from collections import Counter


# Define a class Node to represent each node in the decision tree
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature      # Feature index used for splitting
        self.threshold = threshold  # Threshold value for the split
        self.left = left            # Left child node
        self.right = right          # Right child node
        self.value = value          # Value for leaf node (class label)

        
    def is_leaf_node(self):
        return self.value is not None  # Check if the node is a leaf node (no further splits)
    

# Define the DecisionTree class for the decision tree classifier
class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split  # Minimum samples required to split
        self.max_depth = max_depth                 # Maximum depth of the tree
        self.n_features = n_features               # Number of features to consider for splitting
        self.root = None                           # Root of the decision tree

        
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
       
    # Set n_features to number of features in X if not specified, otherwise use specified or min of both
        self.root = self._grow_tree(X, y)  # Start growing the tree from the root

        
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape              # Get the number of samples and features
        n_labels = len(np.unique(y))              # Get the number of unique labels

        # Check the stopping criteria for recursion
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)  # Get the most common label in current node
            return Node(value=leaf_value)            # Return a leaf node

        
        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)  # Randomly select features to consider

        
        # Find the best feature and threshold for splitting
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # Create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)    # Split the data
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)        # Recursively grow left subtree
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)     # Recursively grow right subtree
        return Node(best_feature, best_thresh, left, right)                     # Return the current node

    
    def _best_split(self, X, y, feat_idxs):
        best_gain = -1  # Initialize best gain to a negative value
        split_idx, split_threshold = None, None  # Initialize best feature index and threshold

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]          # Get the feature values for the current feature index
            thresholds = np.unique(X_column)   # Get unique values in the feature column

            for thr in thresholds:
                # Calculate the information gain for each threshold
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:           # Update best gain if current gain is better
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold      # Return the best feature index and threshold

    
    def _information_gain(self, y, X_column, threshold):
        parent_entropy = self._entropy(y)     # Calculate the entropy of the parent node

        # Create child nodes
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:  # If no split, return 0 gain
            return 0

        # Calculate the weighted average entropy of the children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r

        # Calculate the information gain
        information_gain = parent_entropy - child_entropy
        return information_gain

    
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()  # Indices of samples <= threshold
        right_idxs = np.argwhere(X_column > split_thresh).flatten()  # Indices of samples > threshold
        return left_idxs, right_idxs                                 # Return the indices of the split

    
    def _entropy(self, y):
        hist = np.bincount(y)                                     # Count the occurrences of each class
        ps = hist / len(y)                                        # Calculate the probabilities
        return -np.sum([p * np.log(p) for p in ps if p > 0])      # Calculate and return the entropy

    
    def _most_common_label(self, y):
        counter = Counter(y)                                      # Count the occurrences of each label
        value = counter.most_common(1)[0][0]                      # Get the most common label
        return value

    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])  # Traverse the tree for each sample

    
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():                               # If the node is a leaf, return its value
            return node.value

        if x[node.feature] <= node.threshold:                 # Traverse the left or right subtree based on the feature value
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


# Training

In [15]:
# Importing required libraries and modules
from sklearn import datasets                          # Importing datasets module from scikit-learn
from sklearn.model_selection import train_test_split  # Importing train_test_split function from model_selection module
import numpy as np                                    # Importing numpy library and aliasing it as np
from DecisionTree import DecisionTree                 # Importing DecisionTree class from DecisionTree module


# Loading the Breast Cancer dataset
data = datasets.load_breast_cancer()                  # Loading the Breast Cancer dataset


# Splitting the dataset into features (X) and target variable (y)
X, y = data.data, data.target                        # Assigning features (X) and target variable (y) from the dataset


# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234
)  
# Splitting X and y into train and test subsets with 80% for training and 20% for testing


# Creating an instance of the DecisionTree class with a maximum depth of 10
clf = DecisionTree(max_depth=10)                   # Creating an instance of the DecisionTree class with max_depth=10


# Training the Decision Tree classifier on the training data
clf.fit(X_train, y_train)                         # Fitting the classifier on the training data


# Making predictions on the test data
predictions = clf.predict(X_test)                 # Predicting the labels for the test data using the trained classifier


# Defining a function to calculate accuracy
def accuracy(y_test, y_pred):
    # Calculating the accuracy as the proportion of correct predictions
    return np.sum(y_test == y_pred) / len(y_test)


# Calculating the accuracy of the model on the test data
acc = accuracy(y_test, predictions)             # Calculating the accuracy of the model
print(acc)                                      # Printing the accuracy of the model


0.9210526315789473
